In [1]:
import pandas as pd
from pymongo import MongoClient

In [9]:
df = pd.read_csv("/home/centos/SNP_DB/dbSNP154_chr1_0_MUSC_AnnotationResults.txt", sep="\t")

KeyboardInterrupt: 

In [4]:
df.columns

Index(['#Chr', 'Start', 'End', 'Ref', 'Alt', 'RSID', 'AF', 'Region_Ensembl',
       'GeneName_ID_Ensembl', 'GeneInfo_DistNG_Ensembl', 'ExonicFunc_Ensembl',
       'AAChange_Ensembl', 'Promoter_like_region', 'GWAS_Catalog', '1000g_ALL',
       '1000g_EAS', '1000g_AMR', '1000g_AFR', '1000g_EUR', '1000g_SAS',
       'gnomAD_All', 'gnomAD_EAS', 'gnomAD_AMR', 'gnomAD_AFR', 'gnomAD_NFE',
       'gnomAD_SAS', 'chromHMM_E026_25', 'OpenChromatin_hMSC',
       'ATAC-MBPS_hMSC-Day1', 'SigHiC_hMSC', 'chromHMM_E129_25',
       'OpenChromatin_OB', 'ATAC-MBPS_hMSC-Day15', 'SigHiC_OB13', 'SigHiC_OC'],
      dtype='object')

In [5]:
df.dtypes

#Chr                         int64
Start                        int64
End                          int64
Ref                         object
Alt                         object
RSID                        object
AF                          object
Region_Ensembl              object
GeneName_ID_Ensembl         object
GeneInfo_DistNG_Ensembl     object
ExonicFunc_Ensembl          object
AAChange_Ensembl            object
Promoter_like_region        object
GWAS_Catalog                object
1000g_ALL                  float64
1000g_EAS                  float64
1000g_AMR                  float64
1000g_AFR                  float64
1000g_EUR                  float64
1000g_SAS                  float64
gnomAD_All                 float64
gnomAD_EAS                 float64
gnomAD_AMR                 float64
gnomAD_AFR                 float64
gnomAD_NFE                 float64
gnomAD_SAS                 float64
chromHMM_E026_25            object
OpenChromatin_hMSC          object
ATAC-MBPS_hMSC-Day1 

In [6]:
df['SigHiC_hMSC'][20]

'RegulatoryBin:1:10000:12000;PromoterBin:1:16608000:16610000,1:16612000:16614000'

In [7]:
df['SigHiC_hMSC'].nunique()

15883

In [8]:
df.shape

(12206857, 35)

In [9]:
df['RSID'][0]

'rs1570391677'

In [10]:
df['RSID'][2]

'rs1570391694'

In [11]:
list(df.iloc[1])

[1,
 10002,
 10002,
 'A',
 'C',
 'rs1570391692',
 'KOREAN:0.005597',
 'intergenic',
 'NONE,DDX11L1',
 'dist=NONE;dist=1867',
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 '25_Quies',
 nan,
 nan,
 'RegulatoryBin:1:10000:12000;PromoterBin:1:16608000:16610000,1:16612000:16614000',
 '25_Quies',
 nan,
 nan,
 nan,
 nan]

In [12]:
df.iloc[1]

#Chr                                                                       1
Start                                                                  10002
End                                                                    10002
Ref                                                                        A
Alt                                                                        C
RSID                                                            rs1570391692
AF                                                           KOREAN:0.005597
Region_Ensembl                                                    intergenic
GeneName_ID_Ensembl                                             NONE,DDX11L1
GeneInfo_DistNG_Ensembl                                  dist=NONE;dist=1867
ExonicFunc_Ensembl                                                       NaN
AAChange_Ensembl                                                         NaN
Promoter_like_region                                                     NaN

In [13]:
df[~pd.isna(df.SigHiC_hMSC)]

MemoryError: Unable to allocate array with shape (9, 8446476) and data type object

MemoryError: Unable to allocate array with shape (7, 8446476) and data type object

In [3]:
client = MongoClient('localhost', 27019)

In [4]:
db = client.TableView

In [6]:
collection = db["dbSNP_Annotation"]# promtoer_hMSC

In [10]:
def process(df):
    for i, r in df.iterrows():
        document = {c: r[c] for c in df.columns}
        collection.insert_one(document)
        if i % 100000 == 0:
            print(i)

In [ ]:
chunksize = 10 ** 6
cnt = 0
for chunk in pd.read_csv("/home/centos/SNP_DB/dbSNP154_chr1_0_MUSC_AnnotationResults.txt", sep="\t", chunksize=chunksize):
    try:
        process(chunk)
        cnt += 1
    except:
        print(cnt)
        print("wrong format")
            

/home/centos/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (9,10,11,12,13,27,28,29,31,32,33,34) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


0
100000
200000
300000
400000
500000
600000
700000
800000
900000


/home/centos/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (10,11,12,13,27,28,29,32,33,34) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


1000000
1100000
1200000
1300000


In [ ]:
update = {"$rename": {"gwasCatalog_202007":"gwasCatalog_202012"}}
collection.update_many({}, update)

In [ ]:
{"Disease":"XXXX", "SNPs":["rs12312312", "rs23232"]}

In [7]:
for doc in collection.find({"RSID":"rs1226858834"}):
    print(doc)


In [48]:
collection.create_index("RSID")

'RSID_1'

In [19]:
collection.count({})

<ipython-input-19-9fd663721246>:1: DeprecationWarning: count is deprecated. Use estimated_document_count or count_documents instead. Please note that $where must be replaced by $expr, $near must be replaced by $geoWithin with $center, and $nearSphere must be replaced by $geoWithin with $centerSphere
  collection.count({})


ServerSelectionTimeoutError: localhost:27019: [Errno 111] Connection refused, Timeout: 30s, Topology Description: <TopologyDescription id: 6000969c272ff8cb43ab3dd1, topology_type: Single, servers: [<ServerDescription ('localhost', 27019) server_type: Unknown, rtt: None, error=AutoReconnect('localhost:27019: [Errno 111] Connection refused')>]>

In [ ]:
db